In [0]:
filePath = "dbfs:/FileStore/GlobalRetail/bronze_layer/product_catalog/products.json"

In [0]:
df = spark.read.json(filePath)
df.display()

### Incremental file load

In [0]:
from pyspark.sql.functions import current_timestamp

*CREATING A TIMESTAMP FOR THE CURRENT INGESTION TIME*

In [0]:
df_new = df.withColumn("ingestion_timestamp", current_timestamp())

In [0]:
df_new.display()

### Saving as a delta table

In [0]:
# ensure using the correct database
spark.sql("USE globalretail_bronze")

# writing the database to a delta table in the bronze layer
df_new.write.format("delta")\
            .mode('append')\
            .saveAsTable("bronze_product")

In [0]:
spark.sql("SELECT * FROM bronze_product LIMIT 10").display()

### Removing the file from main folder to archive folder

In [0]:
import datetime

In [0]:
archive_folder = "dbfs:/FileStore/GlobalRetail/bronze_layer/product_catalog/archive/"
archive_filepath = archive_folder + "_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
print(archive_filepath)

In [0]:
# moving the file to the archive folder
# filePath = "dbfs:/FileStore/GlobalRetail/bronze_layer/product_catalog/archive_2025-02-08_20-03-10"
dbutils.fs.mv(filePath, archive_filepath)